# ⚡ Energy Usage Forecasting with Prophet
This notebook demonstrates a simple univariate time series forecasting workflow using Facebook Prophet.

We will use historical energy consumption data to forecast future demand.

In [ ]:
# 📦 Install dependencies (uncomment if running in a fresh environment)
# !pip install pandas matplotlib prophet

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet

## 🔹 Load Dataset
Replace `energy.csv` with the actual dataset you're using (e.g., Open Power System Data).

In [ ]:
df =  pd.read_csv("time_series_60min_singleindex.csv", parse_dates=["utc_timestamp"])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.rename(columns={'timestamp': 'ds', 'load': 'y'})
df = df.sort_values('ds')
df.head()

## 📉 Visualize the time series

In [ ]:
df.set_index('ds')['y'].plot(figsize=(12, 4), title='Energy Consumption');

## 📈 Forecast with Prophet

In [ ]:
model = Prophet()
model.fit(df)

future = model.make_future_dataframe(periods=30)  # Forecast 30 days ahead
forecast = model.predict(future)

## 📊 Visualize forecast with confidence intervals

In [ ]:
fig1 = model.plot(forecast)
fig2 = model.plot_components(forecast)

## 🚨 Anomaly Detection (Simple Residuals)

In [ ]:
df_merged = df.merge(forecast[['ds', 'yhat']], on='ds')
df_merged['residual'] = df_merged['y'] - df_merged['yhat']
threshold = 2 * df_merged['residual'].std()
df_merged['anomaly'] = df_merged['residual'].abs() > threshold

plt.figure(figsize=(12, 4))
plt.plot(df_merged['ds'], df_merged['y'], label='Actual')
plt.plot(df_merged['ds'], df_merged['yhat'], label='Forecast')
plt.scatter(df_merged['ds'][df_merged['anomaly']], df_merged['y'][df_merged['anomaly']],
            color='red', label='Anomaly')
plt.legend();